<a href="https://colab.research.google.com/github/rikdantas/Algoritmos-Estruturas-Dados-II/blob/main/week02/week02_datastructure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Desafio: Autocompletar Palavras**
### Aluno: Paulo Ricardo Dantas
### Matrícula: 20230000789

### Objetivo:
**Criar um sistema de autocompletar palavras usando uma árvore AVL.**
### Etapas:
- Pré-processamento do Corpus:

- Construção da Árvore AVL:

- Autocompletar:

- Análise de Desempenho:


In [53]:
#Importando bibliotecas
import plotly.graph_objects as go
import string
import time
import plotly.express as px

## [1/4] Pré-processamento do Corpus
* Converta todo o texto para minúsculas.
* Remova pontuação e caracteres especiais.
* Divida o texto em palavras.
* Remova palavras de parada (ex: "e", "ou", "mas", "a", "o" etc.) se desejar.

In [54]:
# Eu pedi para que o ChatGPT me gerasse um texto de 30 linhas. Esse é o que vou usar como corpus do programa.
# Salvei o arquivo em txt e vou deixar disponibilizado na pasta do git.

# Lembrar de fazer o upload do corpus para o colab para o código funcionar

with open('corpus.txt', "r", encoding="utf-8") as corpus: # Guardando o corpus numa variável
    texto = corpus.read()

# Primeiro vou separar as palavras em uma lista
palavras = texto.split()


In [55]:
# Removendo as pontuações

def remove_pontuacoes(texto):
    translator = str.maketrans('', '', string.punctuation)
    return texto.translate(translator)

# Percorre a lista e remove todas as pontuações de cada string
palavras = [remove_pontuacoes(palavra) for palavra in palavras]

In [56]:
# Deixando todas as palavras em minúsculo
palavras = [palavra.lower() for palavra in palavras]

In [57]:
# Para finalizar a parte de pré processamento, vou usar uma lista auxiliar para remover itens repetidos.
aux = [] # Vazio

# O laço vai verificar se cada palavra já está na lista. Se não estiver, é adicionada à lista. Se já estiver na lista, pula para a próxima palavra.

for palavra in palavras:
    if palavra not in aux:
        aux.append(palavra)

# Por fim eu vou guardar os valores dessa nova lista na minha antiga, para fins de organização.
palavras = aux


##[2/4] Construção da Árvore AVL
* Insira todas as palavras únicas do corpus na árvore AVL.

**OBS: Para construção da ALV vai ser reutilizado o código disponibilizado pelo professor Ivanovitch**

**OBS2: A fim de tentar enxugar o código, foi retirado a documentação do código do professor**

In [58]:
class Node:

    def __init__(self, value):

        self.value = value
        self.left_child = None
        self.right_child = None


class BST:
    def __init__(self):

        self.root = None

    def add(self, value):
        if self.root is None:
            self.root = Node(value)
        else:
            self._add_recursive(self.root, value)

    def _add_recursive(self, current_node, value):

        if value <= current_node.value:
            if current_node.left_child is None:
                current_node.left_child = Node(value)
            else:
                self._add_recursive(current_node.left_child, value)
        else:
            if current_node.right_child is None:
                current_node.right_child = Node(value)
            else:
                self._add_recursive(current_node.right_child, value)

    def _contains(self, current_node, value):

        if current_node is None:
            return False
        if current_node.value == value:
            return True
        if value < current_node.value:
            return self._contains(current_node.left_child, value)
        return self._contains(current_node.right_child, value)

    def contains(self, value):
        return self._contains(self.root, value)

    def get_height(self):
        return self._get_height(self.root)

    def _get_height(self, node):
        if node is None:
            return 0
        left_height = self._get_height(node.left_child)
        right_height = self._get_height(node.right_child)

        return max(left_height, right_height) + 1

In [59]:
class AVLNode(Node):
    def __init__(self, value):
        super().__init__(value)
        self.height = 1
        self.imbalance = 0

    def calculate_height_and_imbalance(self):
        # Calculate the height of the left child subtree
        left_height = 0
        if self.left_child is not None:
            left_height = self.left_child.height

        # Calculate the height of the right child subtree
        right_height = 0
        if self.right_child is not None:
            right_height = self.right_child.height

        # Update the height of this node
        self.height = 1 + max(left_height, right_height)

        # Calculate and update the imbalance factor for this node
        self.imbalance = left_height - right_height


In [60]:
class AVLTree(BST):
    def __init__(self):
        super().__init__()

    def add(self, value):
        self.root = self._add_recursive(self.root, value)  # Note that self.root is updated here


    def _add_recursive(self, current_node, value):
        # If the current node is None, return a new AVLNode containing the value
        if current_node is None:
            return AVLNode(value)

        # Check if current_node is of the base class Node and cast it to AVLNode if necessary
        # This is necessary to not change the add() in the BST class.
        # When the first node is added, the type of the root is Node, so we need to cast it
        if isinstance(current_node, Node) and not isinstance(current_node, AVLNode):
            current_node = AVLNode(current_node.value)
            current_node.left_child = self.root.left_child
            current_node.right_child = self.root.right_child
            self.root = current_node

        # Determine whether the value should be inserted to the left or right subtree
        if value <= current_node.value:
            current_node.left_child = self._add_recursive(current_node.left_child, value)
        else:
            current_node.right_child = self._add_recursive(current_node.right_child, value)

        # Update the height and imbalance factor for the current node
        current_node.calculate_height_and_imbalance()

        # Check if tree balancing is needed and balance if necessary
        if abs(current_node.imbalance) == 2:
            return self._balance(current_node)

        return current_node

    def get_height(self):
        if self.root is None:
            return 0
        return self.root.height

    def _rotate_left(self, node):
        # Store the pivot (the root of the right subtree of 'node')
        pivot = node.right_child

        # Update the right child of 'node' to be the left child of the pivot
        node.right_child = pivot.left_child

        # Set the left child of the pivot to be the node
        pivot.left_child = node

        # Recalculate the height and imbalance factor for the rotated node
        node.calculate_height_and_imbalance()

        # Recalculate the height and imbalance factor for the pivot
        pivot.calculate_height_and_imbalance()

        # Return the pivot as the new root of this subtree
        return pivot


    def _rotate_right(self, node):
        # Store the pivot (the root of the left subtree of 'node')
        pivot = node.left_child

        # Update the left child of 'node' to be the right child of the pivot
        node.left_child = pivot.right_child

        # Set the right child of the pivot to be the node
        pivot.right_child = node

        # Recalculate the height and imbalance factor for the rotated node
        node.calculate_height_and_imbalance()

        # Recalculate the height and imbalance factor for the pivot
        pivot.calculate_height_and_imbalance()

        # Return the pivot as the new root of this subtree
        return pivot

    def _balance(self, node):
      # Case 1: Left subtree is higher than right subtree
      if node.imbalance == 2:
          pivot = node.left_child
          # Single right rotation
          if pivot.imbalance == 1:
              return self._rotate_right(node)
          # Double rotation: Left-Right
          else:
              node.left_child = self._rotate_left(pivot)
              return self._rotate_right(node)
      # Case 2: Right subtree is higher than left subtree
      else:
          pivot = node.right_child
          # Single left rotation
          if pivot.imbalance == -1:
              return self._rotate_left(node)
          # Double rotation: Right-Left
          else:
              node.right_child = self._rotate_right(pivot)
              return self._rotate_left(node)

    def procuraPalavras(self, prefixo):
        result = []
        self._procuraPalavras(self.root, prefixo, result)
        return result

    def _procuraPalavras(self, current_node, prefixo, result):
        if current_node is None:
            return

        if current_node.value.startswith(prefixo):
            result.append(current_node.value)

        if current_node.value >= prefixo:
            self._procuraPalavras(current_node.left_child, prefixo, result)

        self._procuraPalavras(current_node.right_child, prefixo, result)

In [61]:
# Depois de usar todas as classes e implementação feita pelo professor, o próximo passo é passar nosso corpus para a árvore AVL

avl = AVLTree() # Iniciando a árvore
for palavra in palavras: # Usando um laço para percorrer a lista e adicionar os valores a árvore
    avl.add(palavra)

In [62]:
# Para visualizar a árvore e ver se o trecho acima funcionou, vou utilizar uma função também disponibilizada pelo professor Ivanovitch para visualização da AVL.

def visualize_avl_tree(avl_tree):

    # Lists to store node values, hover texts, x and y coordinates, and edge coordinates
    node_values = []
    hover_texts = []
    Xn, Yn = [], []
    Xe, Ye = [], []

    # Traversing the tree to collect node and edge data
    def traverse(node, x=0, y=0, layer=1):
        if node is not None:
            node_values.append(node.value)
            hover_texts.append(f"Value: {node.value}Height: {node.height}Imbalance: {node.imbalance}")
            Xn.append(x)
            Yn.append(y)
            if node.left_child:
                Xe.extend([x, x - layer])
                Ye.extend([y, y - layer * 2])
                traverse(node.left_child, x=x-layer, y=y-layer*2, layer=layer/2)
            if node.right_child:
                Xe.extend([x, x + layer])
                Ye.extend([y, y - layer * 2])
                traverse(node.right_child, x=x+layer, y=y-layer*2, layer=layer/2)

    # Initialize traversal with the root node of the AVL tree
    traverse(avl_tree.root)

    # Create Plotly figure
    fig = go.Figure()

    # Add nodes (as scatter plot)
    fig.add_trace(go.Scatter(x=Xn,
                             y=Yn,
                             mode='markers+text',
                             name='Nodes',
                             marker=dict(symbol='circle-dot',
                                         size=30,
                                         color='blue'),
                             text=node_values,  # This will appear inside the node
                             hoverinfo='text',
                             hovertext=hover_texts,  # This will appear upon hover
                             textposition='top center'))

    # Add edges (as line plot)
    fig.add_trace(go.Scatter(x=Xe,
                             y=Ye,
                             mode='lines',
                             name='Edges',
                             line=dict(width=1.5, color='gray')))

    # Set layout properties
    fig.update_layout(showlegend=False,
                      hovermode='closest',
                      xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                      yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))

    fig.show()

In [63]:
visualize_avl_tree(avl)

##[3/4] Autocompletar
* Implemente uma função que percorre a árvore AVL e retorna palavras que
começam com um determinado prefixo. A função deve parar de buscar na árvore
assim que encontrar um nó que não corresponda ao prefixo.

**PARA O FUNCIONAMENTO DO PROGRAMA, ESSAS FUNÇÃO PRECISA SER INSERIDA DENTRO DA CLASSE AVL, ENTÃO VOU COPIAR A FUNÇÃO QUE FOI ADICIONADA NA CLASSE AVL E ADICIONAR COMENTÁRIOS**



```
def procuraPalavras(self, prefixo):
    # Cria uma lista vazia para armazenar as palavras que começam com a prefixo
    result = []

    # Chama a função recursiva para iniciar a busca a partir da raiz da árvore
    self._procuraPalavras(self.root, prefixo, result)

    # Retorna a lista de palavras encontradas
    return result
def _procuraPalavras(self, current_node, prefixo, result):
    # Verifica se o nó atual é nulo (caso base para a recursão)
    if current_node is None:
        return

    # Verifica se o valor do nó começa com a prefixo desejada
    if current_node.value.startswith(prefixo):
        # Se sim, adiciona o valor do nó à lista de palavras encontradas
        result.append(current_node.value)

    # Verifica se o valor do nó atual é maior ou igual à prefixo
    if current_node.value >= prefixo:
        # Se for, continua a busca na subárvore esquerda
        self._procuraPalavras(current_node.left_child, prefixo, result)
        
    # Sempre continua a busca na subárvore direita
    self._procuraPalavras(current_node.right_child, prefixo, result)
```



In [64]:
# Por fim vamos ter o programa funcionando como foi descrito na atividade.
prefixo = input("Digite um prefixo para saber se tem palavras que assim se iniciam: ")
saida = avl.procuraPalavras(prefixo)
print(saida)

Digite um prefixo para saber se tem palavras que assim se iniciam: i
['impactos', 'igualitário', 'isso', 'independentemente', 'inclusiva', 'importante', 'internet', 'instituições', 'infraestrutura', 'investimentos']


##[4/4] Análise de Desempenho
* Compare o desempenho (tempo de inserção, altura da árvore, tempo de busca)
da árvore AVL com uma estrutura de dados mais simples, como uma lista ou
uma árvore binária de busca sem balanceamento.
* Analise o impacto do tamanho do corpus no desempenho da árvore AVL.

### Altura da AVL vs Altura da BST
A primeira etapa da análise, vai ser comparar a altura da árvore AVL com a altura da árvore BST, inserindo os objetos na mesma ordem

In [65]:
# Iniciando a árvore BST
bst = BST() # Iniciando a árvore
for palavra in palavras: # Usando um laço para percorrer a lista e adicionar os valores a árvore
    bst.add(palavra)

O seguinte trecho de código foi adicionado lá na classe BST, para retornar a altura da árvore BST.

```
def get_height(self):
        # Chama a função auxiliar _get_height para calcular a altura a partir da raiz
        return self._get_height(self.root)

    def _get_height(self, node):
        # Caso base: se o nó for nulo, a altura é 0
        if node is None:
            return 0
        
        # Calcula a altura da subárvore esquerda e direita
        left_height = self._get_height(node.left_child)
        right_height = self._get_height(node.right_child)

        # Retorna a altura máxima entre as subárvores esquerda e direita + 1 para contar o próprio nó
        return max(left_height, right_height) + 1
```



In [66]:
bstHeight = bst.get_height()
avlHeight = avl.get_height()
print("A altura da árvore AVL é "+ str(avlHeight) + " e a altura da BST é " + str(bstHeight))

A altura da árvore AVL é 9 e a altura da BST é 17


### Tempo de busca na AVL vs Tempo de busca na lista
Por fim, nessa etapa vai ser comparado o tempo de busca por um item na AVL com o tempo de busca na lista.

OBS: Nesse trecho, vai ser usado um código disponibilizado pelo professor Ivanovitch.

In [83]:
# Generate experiment numbers (replace this with your actual experiment numbers)
experiment_numbers = palavras

times_list = []
times_avl = []

for v in palavras:
    # Measure runtime for list
    start = time.time()
    v in palavras
    end = time.time()
    times_list.append(end - start)

    # Measure runtime for AVL
    start = time.time()
    avl.contains(v)
    end = time.time()
    times_avl.append(end - start)

# Convert to milliseconds for better visualization
times_list_ms = [t * 1000 for t in times_list]
times_avl_ms = [t * 1000 for t in times_avl]

# Create a Plotly figure as a scatter plot

fig = px.scatter(x=[experiment_numbers], y=times_list_ms, labels={"x": "Experiment Number", "y": "Runtime (ms)"},
                 title="Runtime Comparison: List vs. AVL", template="plotly_dark")
fig.add_scatter(x=experiment_numbers, y=times_avl_ms, mode='markers', name='AVL Tree', marker=dict(size=8))

# Update axes properties
fig.update_xaxes(showline=True, linewidth=1, linecolor="gray")
fig.update_yaxes(showline=True, linewidth=1, linecolor="gray")
# Change the name in the legend
fig.update_traces(name="List", selector=dict(name="wide_variable_0"))

# Show the figure
fig.show()


Note que dependendo da execução o resultado do gráfico pode mudar, mas teoricamente a AVL vai ser melhor em média no tempo de execução.